In [1]:
import os

import numpy as np
import pandas as pd

from data_loader import label_str2int_dict

rationale_names = {'td','cs'}
current_path = os.getcwd()
print(current_path)
os.chdir('C:\\Users\\lyq\\DataSet\\FakeNews\\twitter')
print(os.getcwd())



C:\Users\lyq\DataSet
C:\Users\lyq\DataSet\FakeNews\twitter


In [2]:
df = pd.read_csv('twitter.csv',encoding='utf-8')
df.rename(columns={'post_id':'source_id','post_text':'content'},inplace=True)
# content,label,image_id,source_id,split,td_rationale,td_pred,td_acc,cs_rationale,cs_pred,cs_acc
df['label'] = df['label'].apply(lambda x: label_str2int_dict[x])
df['split'] = np.nan
df = df[['source_id','content','label','split','image_id']]
df.columns

Index(['source_id', 'content', 'label', 'split', 'image_id'], dtype='object')

In [3]:
for rationale_name in rationale_names:
    rationale_df = pd.read_csv(f'{rationale_name}.csv')
    rationale_df = rationale_df.dropna(subset=['authenticity','reason'])
    if rationale_df.shape[0] != df.shape[0]:
        raise Exception(f'{rationale_name} len : {rationale_df.shape[0]} != {df.shape[0]}')
    df = df.merge(rationale_df,on='source_id',how='left')
    df = df.rename(columns={
        'authenticity':f'{rationale_name}_pred',
        'reason': f'{rationale_name}_rationale',
    })
    df[f'{rationale_name}_pred'] = df[f'{rationale_name}_pred'].apply(lambda x: label_str2int_dict[x])
    df[f'{rationale_name}_acc'] = df.apply(lambda x : int(x[f'{rationale_name}_pred']==x['label']) ,axis=1)
    print(df.columns)
    
df
    

Index(['source_id', 'content', 'label', 'split', 'image_id', 'cs_pred',
       'cs_rationale', 'cs_acc'],
      dtype='object')
Index(['source_id', 'content', 'label', 'split', 'image_id', 'cs_pred',
       'cs_rationale', 'cs_acc', 'td_pred', 'td_rationale', 'td_acc'],
      dtype='object')


,source_id,content,label,split,image_id,cs_pred,cs_rationale,cs_acc,td_pred,td_rationale,td_acc
0,325145334739267584,PIC: Comparison of #Boston suspect Sunil Tripa...,1,NaN,boston_fake_23,1,Plausibility: The message suggests a comparis...,1,1,1. The message suggests a comparison between ...,1
1,325152091423248385,I'm not completely convinced that it's this Su...,1,NaN,boston_fake_34,1,Plausibility: The message suggests doubt abou...,1,1,1. The message is a tweet expressing doubt ab...,1
2,324315545572896768,4chan and the bombing. just throwing it out th...,1,NaN,boston_fake_15,1,Plausibility: The message is vague and lacks ...,1,1,1. The message is extremely brief and lacks c...,1
3,324581777614180352,4chan thinks they found pictures of the bomber...,1,NaN,boston_fake_08,1,Plausibility: Claims from anonymous sources l...,1,1,1. Source: The source of the message is 4chan...,1
4,324665423956176896,"Ola ke ase, investigando las bombas de Boston ...",1,NaN,boston_fake_35,1,Plausibility: The message is written in a mix...,1,1,1. Language: The message is written in a mix ...,1
...,...,...,...,...,...,...,...,...,...,...,...
14190,700662094107119616,14 children! 14 different fathers! All of them...,1,NaN,woman_14_children_1,1,Plausibility: The claim that a woman has 14 c...,1,1,1. The claim that a woman has 14 children wit...,1
14191,700638809155710977,Meet The Woman Who Has Given Birth To 14 Child...,1,NaN,woman_14_children_1,1,Plausibility: While it is possible for a woma...,1,1,1. Sensationalism: The headline is highly sen...,1
14192,700618091886047232,"RT @Viasat1Ghana: Woman, 36, gives birth to 14...",1,NaN,woman_14_children_1,1,Plausibility: While it is possible for a woma...,1,1,1. **Source**: The source of the tweet is Via...,1
14193,700569502874992640,Woman Breaks World Record With 14 Children fro...,1,NaN,woman_14_children_2,1,Plausibility: While it is biologically possib...,1,1,1. **Source**: The source of the message is n...,1


In [4]:

df.to_csv('twitter_llm_rationales.csv',index=False)

In [6]:
print(f"sum: {df.shape[0]}")
value_count = df['label'].value_counts()
print(f"real data: {value_count.get(0,0)}")
print(f"fake data: {value_count.get(1,0)}")

sum: 14195
real data: 6432
fake data: 7763


In [7]:
def filter_data(df):
    for r_name in rationale_names:
        df = df[ (df[f'{r_name}_pred'] != -1) & (df[f'{r_name}_rationale'] is not None) ]
    return df
        
print(f"before filtering : {df.shape[0]}  real data :{(df['label']==1).sum()} fake data: {(df['label']==0).sum()}")
df = filter_data(df)
print(f"after filtering : {df.shape[0]}  real data :{(df['label']==1).sum()} fake data: {(df['label']==0).sum()}")


before filtering : 14195  real data :7763 fake data: 6432
after filtering : 14195  real data :7763 fake data: 6432


In [8]:
from Util import cal_rationale_metrics
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score

for r_name in rationale_names:
    pred = df[r_name+ "_pred"].tolist()
    label = df['label'].tolist()
    print(cal_rationale_metrics(pred,label))
    

{'acc': 0.6779852060584713, 'recall': np.float64(0.6724966301911799), 'recall_real': np.float64(0.6139614427860697), 'recall_fake': np.float64(0.7310318175962901), 'precision': np.float64(0.6748845163901629), 'precision_real': np.float64(0.6541328474407818), 'precision_fake': np.float64(0.695636185339544), 'f1_macro': np.float64(0.6731528887955175), 'f1_real': np.float64(0.6334108589301468), 'f1_fake': np.float64(0.7128949186608882)}
{'acc': 0.6865797816132441, 'recall': np.float64(0.6794081128878344), 'recall_real': np.float64(0.6029228855721394), 'recall_fake': np.float64(0.7558933402035296), 'precision': np.float64(0.6842472037243117), 'precision_real': np.float64(0.671747791442924), 'precision_fake': np.float64(0.6967466160056993), 'f1_macro': np.float64(0.6802965557120234), 'f1_real': np.float64(0.6354772634166326), 'f1_fake': np.float64(0.7251158480074142)}


In [9]:
def split_data0(df,train_ratio=0.8,valid_ratio=0.1):
    train_nums = int(df.shape[0]*train_ratio)
    valid_nums = int(df.shape[0]*valid_ratio)
    return df.iloc[:train_nums],df.iloc[train_nums:train_nums+valid_nums],df.iloc[train_nums+valid_nums:]

def split_data(df,train_ratio=0.8,valid_ratio=0.1):
    real_data = df[df['label'] == 1]
    fake_data = df[df['label'] == 0]
    print(f"real_data count : {real_data.shape[0]}")
    print(f"fake_data count : {fake_data.shape[0]}")
    train_real,valid_real,test_real = split_data0(real_data,train_ratio,valid_ratio)
    train_fake,valid_fake,test_fake = split_data0(fake_data,train_ratio,valid_ratio)
    train_df,valid_df,test_df = pd.concat([train_real,train_fake]),pd.concat([valid_real,valid_fake]),pd.concat([test_real,test_fake])
    train_df['split'] = 'train'
    valid_df['split'] = 'valid'
    test_df['split'] = 'test'
    return train_df,valid_df,test_df


train_df,valid_df,test_df = split_data(df,train_ratio=0.8,valid_ratio=0.1)


train_df.shape[0],valid_df.shape[0],test_df.shape[0]

real_data count : 7763
fake_data count : 6432


(11355, 1419, 1421)

In [10]:
train_df.to_csv('train.csv',index=False)
valid_df.to_csv('val.csv',index=False)
test_df.to_csv('test.csv',index=False)